<a href="https://colab.research.google.com/github/fishfishin/CT-denoise/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import torch
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import skimage
import copy
import cv2
from google.colab import drive
import skimage.external.tifffile


class NLM(nn.Module):
    def __init__(self,  p, s):
        super(NLM, self).__init__()
        self.op = p
        self.s = s
        self.p = p * 2
        self.p +=1
        self.h = torch.ones((1, 1, self.p, self.p), dtype=torch.float64)
        self.w = self.s
        self.s *= 2
        self.m = nn.ReplicationPad2d(self.w)
        self.c = 0.5 * torch.sum(self.h)

    def forward(self, input):
 
        x_stat = self.m(input)
        z_stat = torch.zeros_like(x_stat,  dtype=torch.float64)
        w_sum = torch.zeros_like(z_stat)

        for i in range(int(self.s/2)):
            for j in range(int(self.s/2)):
                    
                    m = nn.ReplicationPad2d((j,  self.s - j, i,  self.s - i))
                    x_disp = m(input)
                    u_n = torch.square(torch.sub(x_stat, x_disp))
                    v_n = F.conv2d(u_n, self.h.cuda(), stride=1, padding=(self.op, self.op))       
                    dist = torch.from_numpy(np.array((pow((self.w - i), 2) + pow((self.w - j), 2) )/2, dtype=np.float64))
                    w_n = torch.exp( torch.add(-dist, torch.div(v_n, self.c)) )
                    z_stat = torch.add(z_stat, torch.mul(x_disp, w_n))
                    w_sum = torch.add(w_sum, w_n)

        z_stat = torch.div(z_stat, w_sum)
      
        return z_stat[:,:, self.w:-self.w, self.w:-self.w]

def divide_patch(fimg) -> np.float64:
    RESOLUTION =256
    PATCH_SIZE = [9,9]
    Patch_num = RESOLUTION**2
    fimgpad = np.zeros((RESOLUTION + PATCH_SIZE[0] -1, RESOLUTION + PATCH_SIZE[0] -1),dtype =np.float64)
    fimgpad[int((PATCH_SIZE[0]+1)/2)-1:RESOLUTION+int((PATCH_SIZE[0]+1)/2)-1,int((PATCH_SIZE[0]+1)/2)-1:RESOLUTION+int((PATCH_SIZE[0]+1)/2)-1]=fimg
    state = np.zeros((Patch_num,PATCH_SIZE[0], PATCH_SIZE[0]))
    count = 0
    for xcord in range(RESOLUTION):
        for ycord in range(RESOLUTION):
            temp=fimgpad[xcord:xcord+PATCH_SIZE[0],ycord:ycord+PATCH_SIZE[0]]
            state[count,:,: ] = temp
            count += 1
    return state

def PSNR(img1, img2):
    D = np.array(img1 - img2, dtype=np.float64)
    D[:, :] = D[:, :]**2
    RMSE = D.sum()/img1.size
    psnr = 10*math.log10(float(1.0**2)/RMSE)
    return psnr

if __name__ == '__main__':
    drive.mount('/content/drive')
    image= skimage.external.tifffile.imread("drive/My Drive/Colab Notebooks/a1.tif")/4095.0
    model = NLM( 2 , 4 ).cuda()
    img = torch.zeros([1,256*256,9,9], dtype=torch.float64)
    img[0,:,:,:] = torch.from_numpy(divide_patch(image[0,:,:]))
    data = torch.zeros((1,1,9,9),dtype=torch.float64)
    for i in range(100):
      print(i)
      for j in range(256*256):
        data[:,:,:,:] = img[:,j,:,:]
        filtered = model(data.cuda())
        img[:,j,:,:] = filtered
        
      f = img.cpu().detach().numpy()
      new = np.reshape(f[0, :, int(9//2), int(9//2)], (256, 256), order='A')
      psnr = PSNR(image[0,:,:], new)
      print( 'psnr: {:.4f}'.format(psnr))
      fig, axs = plt.subplots(1, 1,figsize=(10,10))
      fig.suptitle(" iteration: {}".format(i))
      axs.imshow(new, cmap='gray')
      axs.set_title('f')
      


Mounted at /content/drive
0


KeyboardInterrupt: ignored